## Extract scarps from DEMs

In [3]:
import arcpy
arcpy.env.overwriteOutput = True

# define a function
def ExtractScarps(DEM, filter_size, threshold_value, save_path, output):
    
    # check out spatial analyst
    arcpy.CheckOutExtension("Spatial")
    
    threshold_file = save_path + "junk_threshold_file.tif"
    polygon_file = save_path + "junk_polygon_conversion.shp"
    
    # calculate slope
    slope = arcpy.sa.Slope(DEM)

    # apply range filter to the slope file
    neighborhood = arcpy.sa.NbrCircle(5, "CELL")
    slope_range = arcpy.sa.FocalStatistics(slope, neighborhood, "RANGE", "")

    # extract high values from the range output
    threshold = arcpy.Raster(slope_range) > threshold_value
    threshold.save(threshold_file)

    # convert pixels to polygons with NO simplification
    arcpy.conversion.RasterToPolygon(threshold, polygon_file, "NO_SIMPLIFY")
    
    # extract only gridcode values
    scarp_only = arcpy.analysis.Select(polygon_file, output, '"gridcode" = 1')
    
    # return extension
    arcpy.CheckInExtension("Spatial")